# Postprocessing

In [15]:
import re
import os
import pandas as pd

In [12]:
def read_validation_directory():
    available_content = os.listdir('.') 
    select_pattern = re.compile('^validation-[a-z][a-z][a-z]')
    selected_dir = [content for content in available_content if select_pattern.match(content)]
    return selected_dir

In [34]:
def read_validation_directory_content(validation_directories):
    iteration = 0
    for directory in validation_directories:
        available_content = os.listdir(directory)
        for file in available_content:
            if iteration == 0:
                imported_data = pd.read_csv('{}/{}'.format(directory, file))
                imported_data['model'] = directory[11:]
            else:
                tmp_data = pd.read_csv('{}/{}'.format(directory, file)) 
                tmp_data['model'] = directory[11:] 
                imported_data = pd.concat([imported_data, tmp_data])
            iteration += 1
    
    imported_data.reset_index(drop=True, inplace=True)
    return imported_data

In [ ]:
def text_split(x):
    y = x.split('-')
    return y

In [76]:
def extract_feature_information(df):
    
    df_splitted = df['data'].apply(text_split)
    
    feature_method_list = []
    document_size_list = []
    feature_size_list = []
    ngram_size_list = []
    
    for splitted_entries in df_splitted:
        if splitted_entries[1] == 'idf': # inverse document frequency case
            feature_method_list.append('tf-idf')
            document_size_list.append(splitted_entries[2])
            feature_size_list.append(splitted_entries[3])
            ngram_size_list.append(splitted_entries[4])
        else:
            feature_method_list.append('tf')
            document_size_list.append(splitted_entries[1])
            feature_size_list.append(splitted_entries[2])
            ngram_size_list.append(splitted_entries[3])
    
    df['method'] = feature_method_list
    df['doc'] = document_size_list
    df['feature'] = feature_size_list
    df['ngram'] = ngram_size_list
    df.drop(['data'], axis=1, inplace=True)
    
    
    return df

In [77]:
# read from all validation directories the accuracy and determine model settings
validation_directories = read_validation_directory()
raw_results = read_validation_directory_content(validation_directories)
results = extract_feature_information(raw_results)
results.head()

,accuracy,model,method,doc,feature,ngram
0,0.793279,knn,tf,5000,1000,1
1,0.615071,knn,tf-idf,5000,1000,1
2,0.914460,svc,tf,5000,1000,1


In [81]:
# display results according to accuracy
results.sort_values(by=['accuracy'], ascending=False, inplace=True)
results.head(results.shape[0]) # display all

,accuracy,model,method,doc,feature,ngram
2,0.914460,svc,tf,5000,1000,1
0,0.793279,knn,tf,5000,1000,1
1,0.615071,knn,tf-idf,5000,1000,1
